# Combine Manually Coded and Model Classified Data
## Manual Review, part II

In [21]:
import pandas as pd       # For data analysis
import numpy as np        # For numerical operations
from pathlib import Path  # For writing data and creating directories
import os                 # For reading data files
import config             # For accessing path variables

Read all the CSV data files with manual coding.

In [2]:
coded_path = config.coded_data_path
csvs = os.listdir(coded_path)
print(csvs)

['CHE_sample_no_preds-manually-checked.csv', 'SH_sample_no_preds-manually-checked.csv', 'BXB_sample_no_preds-manually-checked.csv', 'BP_unclassified-manually-checked.csv', 'HL_unclassified-manually-checked.csv', 'WCT_sample_no_preds-manually-checked.csv', 'SW_sample_no_preds-manually-checked.csv', 'OBR_sample_no_preds-manually-checked.csv', 'THS_unclassified-manually-checked.csv']


In [36]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
for csv in csvs:
    if "sample" in csv:
        subdf1 = pd.read_csv(coded_path + csv)
        df1 = pd.concat([df1, subdf1])
    else:
        subdf2 = pd.read_csv(coded_path + csv)
        df2 = pd.concat([df2, subdf2])

In [37]:
df1.rename(columns={
    "Gender bias":"gender_bias", "Unnamed: 1":"index"
    }, inplace=True)
df1.set_index("index", inplace=True)
df1.head(5)

,gender_bias,description_id,eadid,rowid,field,doc
index,,,,,,
11452,NaN,11452,CHE,CHE/01/01,scopecontent,Drafts and meeting notes relating to the creat...
11467,NaN,11467,CHE,CHE/02/04,unittitle,CHE Supporters Review
11471,NaN,11471,CHE,CHE/02/06,unittitle,Friend Newcastle Annual Reports
11554,NaN,11554,CHE,CHE/03/06/12,unittitle,Collection of documents on sex education in sc...
11555,NaN,11555,CHE,CHE/03/06/13,unittitle,"Letters, newsletters and leaflets on the topic..."


In [24]:
df1.gender_bias.value_counts()

gender_bias
Omission - could we say more about his contribution?                                                                        2
Omission - could we say more about her contribution?                                                                        2
Omission - should we say Beverliey Braune?                                                                                  1
Omission - could we list which 'women poets' or the titles of the anthologies?                                              1
Omission - who are 'the contributors'?                                                                                      1
Omission - do we know why it was cancelled?                                                                                 1
Omission - missing Cooper's first name                                                                                      1
Omission - missing Charlotte's maiden name. Do we have any more information about her?                    

In [43]:
df2.rename(columns={
    "Unnamed: 0":"index", "If yeStereotype, Type?":"type", "gender_bias?":"gender_bias"
    }, inplace=True)
df2.set_index("index", inplace=True)
df2.fillna({"gender_bias": "n"}, inplace=True)
df2.head(5)

,doc,gender_bias,type,eadid,description_id,rowid,field
index,,,,,,,
0,'ACERT The First Hundred Days',n,NaN,BP,[186],['BP/09/09/03'],['unittitle']
1,'British Housing: Problems and Solutions',n,NaN,BP,[189],['BP/09/09/05'],['unittitle']
2,'Information Booklet' 1989,n,NaN,BP,[199],['BP/09/09/13'],['unittitle']
3,'Information Booklet' 1990,n,NaN,BP,[203],['BP/09/09/17'],['unittitle']
4,"'Looking Back Moving Forwards', 1985",n,NaN,BP,[193],['BP/09/09/07'],['unittitle']


Standardize the manual coding, with one column for gender bias, one column marking the type of gender bias, and and one column recording any notes about the observed gender bias.

In [38]:
df1["type"] = df1["gender_bias"].str.split(" - ", expand=True)[0]
df1["note"] = df1["gender_bias"].str.rsplit(" - ", expand=True)[1]
df1 = df1[["doc", "gender_bias", "type", "note", "eadid", "description_id", "rowid", "field"]]
df1.head()

,doc,gender_bias,type,note,eadid,description_id,rowid,field
index,,,,,,,,
11452,Drafts and meeting notes relating to the creat...,NaN,NaN,NaN,CHE,11452,CHE/01/01,scopecontent
11467,CHE Supporters Review,NaN,NaN,NaN,CHE,11467,CHE/02/04,unittitle
11471,Friend Newcastle Annual Reports,NaN,NaN,NaN,CHE,11471,CHE/02/06,unittitle
11554,Collection of documents on sex education in sc...,NaN,NaN,NaN,CHE,11554,CHE/03/06/12,unittitle
11555,"Letters, newsletters and leaflets on the topic...",NaN,NaN,NaN,CHE,11555,CHE/03/06/13,unittitle


In [39]:
df1.type.value_counts()

type
Omission      11
Stereotype     1
Name: count, dtype: int64

In [40]:
df1.note.value_counts()

note
could we say more about his contribution?                                                                      2
could we say more about her contribution?                                                                      2
should we say Beverliey Braune?                                                                                1
could we list which 'women poets' or the titles of the anthologies?                                            1
who are 'the contributors'?                                                                                    1
do we know why it was cancelled?                                                                               1
missing Cooper's first name                                                                                    1
missing Charlotte's maiden name. Do we have any more information about her?                                    1
the tone of this biography is different to the others. It is grandiose and frames Reverdy a

In [41]:
# df1.gender_bias.value_counts()
gender_bias_values = list(df1.loc[~df1.gender_bias.isna()].gender_bias)
# print(gender_bias_values[0:2])
df1 = df1.replace(to_replace=gender_bias_values, value="y")
df1 = df1.fillna({"gender_bias": "n"})
df1.tail()

,doc,gender_bias,type,note,eadid,description_id,rowid,field
index,,,,,,,,
43357,First draft typescript of 'Laughter When We're...,n,NaN,NaN,OBR,43357,OBR/3/2/6/16,unittitle
43401,Photocopy of chapter from 'Rules of the Game: ...,n,NaN,NaN,OBR,43401,OBR/3/2/8/6,unittitle
43414,Typescript of talk on 'Translating The Birds f...,n,NaN,NaN,OBR,43414,OBR/3/2/9/7,unittitle
43464,Sheffield Hallam University,n,NaN,NaN,OBR,43464,OBR/7/3,unittitle
43498,Includes flyers and posters for poetry reading...,n,NaN,NaN,OBR,43498,OBR/8/3/3,scopecontent


In [44]:
df2.insert(column="note", value=np.nan, loc=3)
df2.head()

,doc,gender_bias,type,note,eadid,description_id,rowid,field
index,,,,,,,,
0,'ACERT The First Hundred Days',n,NaN,NaN,BP,[186],['BP/09/09/03'],['unittitle']
1,'British Housing: Problems and Solutions',n,NaN,NaN,BP,[189],['BP/09/09/05'],['unittitle']
2,'Information Booklet' 1989,n,NaN,NaN,BP,[199],['BP/09/09/13'],['unittitle']
3,'Information Booklet' 1990,n,NaN,NaN,BP,[203],['BP/09/09/17'],['unittitle']
4,"'Looking Back Moving Forwards', 1985",n,NaN,NaN,BP,[193],['BP/09/09/07'],['unittitle']


In [45]:
df = pd.concat([df1, df2])
df.head()

,doc,gender_bias,type,note,eadid,description_id,rowid,field
index,,,,,,,,
11452,Drafts and meeting notes relating to the creat...,n,NaN,NaN,CHE,11452,CHE/01/01,scopecontent
11467,CHE Supporters Review,n,NaN,NaN,CHE,11467,CHE/02/04,unittitle
11471,Friend Newcastle Annual Reports,n,NaN,NaN,CHE,11471,CHE/02/06,unittitle
11554,Collection of documents on sex education in sc...,n,NaN,NaN,CHE,11554,CHE/03/06/12,unittitle
11555,"Letters, newsletters and leaflets on the topic...",n,NaN,NaN,CHE,11555,CHE/03/06/13,unittitle


In [46]:
df.gender_bias.value_counts()

gender_bias
n    6036
y      35
Name: count, dtype: int64

Only 0.6% of the reviewed descriptions were noted as having gender bias...

In [48]:
df.type.value_counts()

type
Omission                   30
Sterotype                   2
Stereotype and Omission     2
Stereotype                  1
Name: count, dtype: int64

...most of which were *Omission*!

Export the combined manually coded data.

In [49]:
combined_filename = "all_manually_coded_data.csv"
df.to_csv(config.coded_data_path+combined_filename)